In [1]:
import os
import json
import random
import numpy as np
import pandas as pd
import math
import torch
import gpytorch
import sys
import re
import statistics
from matplotlib import pyplot as plt
from scipy.stats import gaussian_kde
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

In [2]:
import simple_gp
from Kidera import kidera

In [3]:
df = pd.read_csv('data/DDG_Dataset.csv')
# df = pd.read_csv('data/DDG_Dataset_mini.csv')
df = df[df['Temperature'] != "'-"].reset_index(drop=True)
df['Temperature'] = df['Temperature'].astype('float64')
ndata = df.shape[0]
print(df.columns)
print(ndata)
df

Index(['Rowid', 'ID', 'Protein Name', 'Mutation', 'Chain', 'PDB',
       'Temperature', 'pH', 'ΔΔG', 'Reference', 'A', 'R', 'N', 'D', 'C', 'Q',
       'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V',
       'ASP', 'PHE', 'GLN', 'LYS', 'ILE', 'TYR', 'GLY', 'ASN', 'ARG', 'LEU',
       'TRP', 'ALA', 'THR', 'VAL', 'HIS', 'CYS', 'GLU', 'MET', 'PRO', 'SER'],
      dtype='object')
8535


,Rowid,ID,Protein Name,Mutation,Chain,PDB,Temperature,pH,ΔΔG,Reference,...,TRP,ALA,THR,VAL,HIS,CYS,GLU,MET,PRO,SER
0,1,1,Tryptophan synthase alpha chain,E49M,A,1WQ5,298.95,7.0,4.60,PMID: 378988,...,0.0,4.0,2.0,4.0,0.0,0.0,0.0,1.0,2.0,2.0
1,2,2,Tryptophan synthase alpha chain,E49Q,A,1WQ5,298.95,7.0,-2.50,PMID: 378988,...,0.0,4.0,2.0,4.0,0.0,0.0,0.0,1.0,2.0,2.0
2,3,3,Endolysin,W138Y,A,2LZM,298.15,2.2,-1.71,PMID: 911878,...,0.0,5.0,4.0,4.0,0.0,0.0,2.0,2.0,1.0,2.0
3,4,6,Cellular tumor antigen p53,C242S,A,1TUP,283.15,7.2,-3.07,PMID: 1203434,...,0.0,0.0,0.0,1.0,2.0,4.0,2.0,3.0,1.0,2.0
4,5,7,Cellular tumor antigen p53,F134L,A,1TUP,283.15,7.2,-4.78,PMID: 1203434,...,0.0,2.0,5.0,3.0,0.0,5.0,4.0,1.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8530,8570,13318,Transcriptional repressor arc,V25A,A,1ARR,298.15,7.5,-0.40,PMID: 7664079,...,0.0,1.0,0.0,3.0,0.0,0.0,4.0,0.0,0.0,1.0
8531,8571,13319,Transcriptional repressor arc,V18A,A,1ARR,298.15,7.5,-0.50,PMID: 7664079,...,1.0,1.0,0.0,3.0,0.0,0.0,2.0,0.0,1.0,0.0
8532,8572,13320,Transcriptional repressor arc,V33A,A,1ARR,298.15,7.5,-2.10,PMID: 7664079,...,1.0,1.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,2.0
8533,8573,13321,Transcriptional repressor arc,S32A,A,1ARR,298.15,7.5,-3.80,PMID: 7664079,...,0.0,1.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,1.0


In [4]:
kidera.symbol_lookup
kidera_table = pd.read_csv('Kidera/kidera.csv', header=None).set_index(0)
kidera_table

,1,2,3,4,5,6,7,8,9,10
0,,,,,,,,,,
ALA,-1.56,-1.67,-0.97,-0.27,-0.93,-0.78,-0.20,-0.08,0.21,-0.48
ARG,0.22,1.27,1.37,1.87,-1.70,0.46,0.92,-0.39,0.23,0.93
ASN,1.14,-0.07,-0.12,0.81,0.18,0.37,-0.09,1.23,1.10,-1.73
ASP,0.58,-0.22,-1.58,0.81,-0.92,0.15,-1.52,0.47,0.76,0.70
CYS,0.12,-0.89,0.45,-1.05,-0.71,2.41,1.52,-0.69,1.13,1.10
GLN,-0.47,0.24,0.07,1.10,1.10,0.59,0.84,-0.71,-0.03,-2.33
GLU,-1.45,0.19,-1.61,1.17,-1.31,0.40,0.04,0.38,-0.35,-0.12
GLY,1.46,-1.96,-0.23,-0.16,0.10,-0.11,1.32,2.36,-1.66,0.46
HIS,-0.41,0.52,-0.28,0.28,1.61,1.01,-1.85,0.47,1.13,1.63


In [5]:
%%time 
around_acid = np.zeros((ndata, 10))
for index, row in df.iterrows():
    # print(index)
    # print(row)
    for acid, r in kidera_table.iterrows():
        # print(acid)
        # print(r)
        # print(row[acid])
        # print(row[acid] * r)
        around_acid[index, :] += row[acid] * r
        # break
    # break
    
around_acid

CPU times: user 45.5 s, sys: 28 ms, total: 45.5 s
Wall time: 45.5 s


array([[-11.27, -15.66,  15.32, ...,   3.88,   6.42,  -5.23],
       [-11.27, -15.66,  15.32, ...,   3.88,   6.42,  -5.23],
       [ -6.78,  -6.53,   6.59, ...,   0.97,   2.29,  -6.09],
       ...,
       [ -5.25,  -0.13,   5.73, ...,   1.19,  -3.48,   1.  ],
       [ -2.96,  -2.31,   5.41, ...,   4.96,  -1.61,  -3.84],
       [-17.09, -19.41,  -7.  , ..., -11.25,   1.04,   4.88]])

In [6]:
data_x = df.iloc[:, :30].drop(columns=['Rowid', 'ID', 'Protein Name', 'Mutation', 'Chain', 'PDB', 'ΔΔG', 'Reference']).to_numpy()
data_x = np.concatenate([data_x, around_acid], 1)
data_y = df['ΔΔG'].to_numpy()
data_x

array([[298.95,   7.  ,   0.  , ...,   3.88,   6.42,  -5.23],
       [298.95,   7.  ,   0.  , ...,   3.88,   6.42,  -5.23],
       [298.15,   2.2 ,   0.  , ...,   0.97,   2.29,  -6.09],
       ...,
       [298.15,   7.5 ,   1.  , ...,   1.19,  -3.48,   1.  ],
       [298.15,   7.5 ,   1.  , ...,   4.96,  -1.61,  -3.84],
       [293.15,   8.5 ,   0.  , ..., -11.25,   1.04,   4.88]])

In [7]:
# print(np.unique(data_x[:, 0]))
# print(len(np.unique(data_x[:, 0])))
# random.choices(sorted(np.unique(data_x[:, 0])), k=200)
# random.choices(range(2), k=200)

In [ ]:
%%time

os.system('mkdir -p data/boruta')
for r in range(data_x.shape[1]):
    tmp_data_x = np.copy(data_x)
    random_list = random.choices(sorted(np.unique(tmp_data_x[:, r])), k=ndata)
    tmp_data_x[:, r] = random_list
    # pred_y, lower_y, upper_y = simple_gp.solve(tmp_data_x, data_y, 5, 5)
    pred_y, lower_y, upper_y = simple_gp.solve(tmp_data_x, data_y, 5, 100)
    tmp_df = df.copy()
    tmp_df['rand_items'] = random_list
    tmp_df['pred_ΔΔG'] = pred_y
    tmp_df['lower_ΔΔG'] = lower_y
    tmp_df['upper_ΔΔG'] = upper_y
    tmp_df.to_csv('data/boruta/pred_DDG_Dataset_gpytorch_Kidera_{}.csv'.format(r))

/opt/anaconda3/lib/python3.7/site-packages/gpytorch/utils/linear_cg.py:240: UserWarning: This overload of addcmul is deprecated:
	addcmul(Tensor input, Number value, Tensor tensor1, Tensor tensor2, *, Tensor out)
Consider using one of the following signatures instead:
	addcmul(Tensor input, Tensor tensor1, Tensor tensor2, *, Number value, Tensor out) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629427478/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  residual = torch.addcmul(residual, -1, alpha, mvms, out=residual)


Iter 10/100 - Loss: 2.29345
Iter 20/100 - Loss: 2.16014
Iter 30/100 - Loss: 2.13267
Iter 40/100 - Loss: 2.11091
Iter 50/100 - Loss: 2.09760
Iter 60/100 - Loss: 2.08293
Iter 70/100 - Loss: 2.07858
Iter 80/100 - Loss: 2.07247
Iter 90/100 - Loss: 2.06962
Iter 100/100 - Loss: 2.07158
Iter 10/100 - Loss: 2.28653
Iter 20/100 - Loss: 2.15688
Iter 30/100 - Loss: 2.12990
Iter 40/100 - Loss: 2.10919
Iter 50/100 - Loss: 2.09390
Iter 60/100 - Loss: 2.08236
Iter 70/100 - Loss: 2.07443
Iter 80/100 - Loss: 2.07074
Iter 90/100 - Loss: 2.07189
Iter 100/100 - Loss: 2.06967
Iter 10/100 - Loss: 2.29321
Iter 20/100 - Loss: 2.15880
Iter 30/100 - Loss: 2.13278
Iter 40/100 - Loss: 2.11550
Iter 50/100 - Loss: 2.09975
Iter 60/100 - Loss: 2.08412
Iter 70/100 - Loss: 2.07688
Iter 80/100 - Loss: 2.07436
Iter 90/100 - Loss: 2.06696
Iter 100/100 - Loss: 2.06578
Iter 10/100 - Loss: 2.28055
Iter 20/100 - Loss: 2.14993
Iter 30/100 - Loss: 2.12146
Iter 40/100 - Loss: 2.09890
Iter 50/100 - Loss: 2.08420
Iter 60/100 - Los